# Main

## Import Libraries

In [1]:
# import library
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import trange

# sklearn
import sklearn
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.feature_selection import RFECV, SelectKBest, r_regression, f_regression
from sklearn.gaussian_process.kernels import Matern, RBF, CompoundKernel, Product, Sum, ExpSineSquared, RationalQuadratic
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, IsolationForest, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, make_scorer, f1_score, confusion_matrix
from sklearn.decomposition import PCA

# boost algorithm
import xgboost as xgb
import catboost as cat
import lightgbm as lgb

# torch
import torch
from torch import nn
from torch.nn import Module, Linear, Dropout
from torch.nn.functional import tanh, softmax, mse_loss, relu, sigmoid, binary_cross_entropy, nll_loss
from torch.optim import Adam, SGD

# bio library
import biosppy
from biosppy import storage
from biosppy.signals import ecg

# import ResNet
from ResNet import ResNet

DATA_DIR = "Data"
RESULT_DIR = "Result"

## Load Features Data

In [2]:
# Load Data
X_train_df = pd.read_csv(os.path.join(DATA_DIR, "X_train.csv"), header=0, index_col=0)
X_test_df = pd.read_csv(os.path.join(DATA_DIR, "X_test.csv"), header=0, index_col=0)
y_train_df = pd.read_csv(os.path.join(DATA_DIR, "y_train.csv"), header=0, index_col=0)

X_train = X_train_df.values
X_test = X_test_df.values
y_train = y_train_df.values.ravel()

In [2]:
X_train_expert = np.load("./Data/X_train_features.npy")
X_train_DNN = np.load("./Data/X_train_DNN_features.npy")
y_train = np.load("./Data/y_train.npy")

X_test_expert = np.load("./Data/X_test_features.npy")
X_test_DNN = np.load("./Data/X_test_DNN_features.npy")

In [3]:
X_train_features = np.concatenate([X_train_expert, X_train_DNN], axis=1)
X_test_features = np.concatenate([X_test_expert, X_test_DNN], axis=1)

## Data Preprocessing

In [4]:
feature_select = SelectKBest(k=100)
feature_select.fit(X_train_features, y_train)
X_train_features = feature_select.transform(X_train_features)
X_test_features = feature_select.transform(X_test_features)

/Users/lanhongyi/opt/anaconda3/envs/aml_project/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [24] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/lanhongyi/opt/anaconda3/envs/aml_project/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


## Train and Test

In [ ]:
# 预测数据
# model = cat.CatBoostClassifier(iterations=2000, learning_rate=0.05, max_depth=5)
# model.fit(X_train_features, y_train)
# result = model.predict(X_test_features)

In [3]:
# load cnn model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cpu_device = torch.device("cpu")
resnet = ResNet(input_channels=1, output_features=32, output_dim=4).to(device)
resnet.load_state_dict(torch.load("./Model/Resnet_32F.pt", map_location=device))
resnet.eval()
print("load resnet model")

# process the data
X_train_raw = np.nan_to_num(X_train[:, :6000], nan=0)
X_train_raw = X_train_raw.reshape((X_train_raw.shape[0], 1, X_train_raw.shape[1]))
X_test_raw = np.nan_to_num(X_test[:, :6000], nan=0)
X_test_raw = X_test_raw.reshape((X_test_raw.shape[0], 1, X_test_raw.shape[1]))

def batch_predict(model: nn.Module, x: np.ndarray, batch_size: int = 64) -> np.ndarray:
    x = torch.from_numpy(x).float().to(device)
    dataset = torch.utils.data.TensorDataset(x)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)

    y_preds = []
    for i, (x_batch,) in enumerate(dataloader):
        y_pred = torch.argmax(torch.softmax(model(x_batch), dim=-1), dim=1).to(cpu_device).detach().numpy()
        y_preds.append(y_pred)
    y_preds = np.concatenate(y_preds, axis=0)
    return y_preds

result = batch_predict(resnet, X_test_raw, batch_size=64)

load resnet model


In [5]:
result_df = pd.DataFrame(result.reshape((-1, 1)), columns=["y"]).reset_index()
result_df.columns = ["id", "y"]
result_df.to_csv("./Result/result_only_dnn.csv", index=False)